In [1]:
def minkowski(a, b, p):
    return sum(abs(a[i] - b[i]) ** p for i in range(len(a))) ** (1 / p)

def manhattan(a, b):
    return minkowski(a, b, 1)

def euclidean(a, b):
    return minkowski(a, b, 2)

def hamming(a, b):
    return sum(int(a[i] != b[i]) for i in len(a))

In [2]:
import numpy as np
from heapq import nlargest
from sklearn.base import BaseEstimator, ClassifierMixin

class MyKNN(BaseEstimator, ClassifierMixin):
    def __init__(self, k=1, metric=euclidean):
        self.k = k
        self.metric = metric
        
    def fit(self, X, y):
        self.classes = sorted(list(set(y)))
        self.X = X
        self.y = y
        
    def predict(self, X):
        return list(map(self.__predict, X))
        
    def __predict(self, x):
        nn_i = [e[0] for e in sorted([(i, x) for i, x in enumerate(self.X)],
                                     key=lambda e: self.metric(e[1], x))]
        knn_i = nn_i[:self.k]
        while True:
            
            last_nn_dist = self.metric(self.X[knn_i[-1]], x)
            next_n_dist = self.metric(self.X[nn_i[len(knn_i)]], x)
            if last_nn_dist == next_n_dist: # distance tie
                knn_i.append(nn_i[len(knn_i)])
                continue
                
            class_counts = [[self.y[i] for i in knn_i].count(c) for c in self.classes]
            top2 = nlargest(2, class_counts)
            if top2[0] == top2[1]: # count tie
                knn_i.append(nn_i[len(knn_i)])
                continue
                
            return np.argmax(class_counts)

In [3]:
from sklearn import datasets
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import classification_report

iris = datasets.load_iris()
X = iris.data
y = iris.target
target_names = iris.target_names

clf = MyKNN(k=10)
y_pred = cross_val_predict(clf, X, y, cv=10)
print(classification_report(y, y_pred, target_names=target_names))

             precision    recall  f1-score   support

     setosa       1.00      1.00      1.00        50
 versicolor       0.94      0.96      0.95        50
  virginica       0.96      0.94      0.95        50

avg / total       0.97      0.97      0.97       150

